O objetivo dessa aula é finalizar a introdução aos conceitos de tidy e formatação de dados e trazer conceitos e exemplos de Joins.

**Tópicos**:
 - **O que é Data Wrangling:**
 - **Definição de Tidy Format** 
 - **Os 5 problemas mais comuns**
     - As Colunas são Valores e Não nomes de Variáveis
     - Múltiplas Unidades Observacionais armazenadas na mesma tabela
     - Múltiplas Variáveis Armazenadas em uma Coluna
     - Variáveis Armazenadas em Colunas e Linhas
     - Uma única unidade Observacional em várias tabelas
 - **Joins**
     - Inner Join
     - Left Join
     - Right Join
     - Outer Join

In [1]:
#Dependencias
import pandas as pd
import numpy as np
#!pip install datetime
import datetime

## Múltiplas Variáveis Armazenadas em uma Coluna

Esse datase contém registros de casos de tuberculose. É uma contagem de casos por País, ano, idade e sexo.

Problemas:
* Algumas colunas contêm múltiplos valores: sexo e idade
* Mistura de 0 e NaN para representar dados faltosos (faz diferença?)

In [2]:
df_tb_raw = pd.read_csv("tb-raw.csv")
df_tb_raw.head()

,country,year,m014,m1524,m2534,m3544,m4554,m5564,m65,mu,f014
0,AD,2000,0.0,0.0,1.0,0.0,0,0,0.0,NaN,NaN
1,AE,2000,2.0,4.0,4.0,6.0,5,12,10.0,NaN,3.0
2,AF,2000,52.0,228.0,183.0,149.0,129,94,80.0,NaN,93.0
3,AG,2000,0.0,0.0,0.0,0.0,0,0,1.0,NaN,1.0
4,AL,2000,2.0,19.0,21.0,14.0,24,19,16.0,NaN,3.0


m014 significa Homens entre 0 e 14 anos, m1524 significa Homens entre 15 e 24 anos e assim por diante. Então, note que temos, de fato, mais de uma variável em uma única coluna.

Inicialmente, vamos desfazer as colunas, criando uma coluna chamada sexo_e_idade. Dessa forma, as colunas não existirão e passarão a ser uma única contendo sexo e idade e seus valores serão registros.

In [3]:
df_tb_raw_tidy=pd.melt(df_tb_raw , id_vars=["country" , "year"] , value_name = "num_casos", var_name = "sexo_idade")

In [4]:
df_tb_raw_tidy.head()

,country,year,sexo_idade,num_casos
0,AD,2000,m014,0.0
1,AE,2000,m014,2.0
2,AF,2000,m014,52.0
3,AG,2000,m014,0.0
4,AL,2000,m014,2.0


Até agora OK. Precisamos separar a coluna sexo_e_idade em duas: sexo, idade.
para isso, precisamos extrair o caractere que identifica o sexo (m ou f) e também a faixa de idade que tem 3 ou 4 números seguidos. Para isso vamos utilizar uma velha conhecida: a regex.

Vamos fazer isso por partes:
* Usaremos ```\D``` para dar match com qualquer caractere que não seja um número (importante dizer que depois que o match ocorre, não há uma busca posterior nesse caso.)
* Usaremos 
* \d = digito \D= tudo oq não é digito-> um caracter

In [7]:
df_tb_raw_tidy['sexo'] = df_tb_raw_tidy["sexo_idade"].str.extract("(\D)")
df_tb_raw_tidy['faixa'] = df_tb_raw_tidy["sexo_idade"].str.extract("(\d+)")

In [8]:
x = df_tb_raw_tidy['faixa'][35]

In [11]:
def criar_faixas(x):
    if x[0] == '0':
        return x[0]+'-'+x[1:]
    else:
        return x [0:2]+'-'+x[2:]

In [12]:
df_tb_raw_tidy['faixa'] = df_tb_raw_tidy['faixa'].astype('object')

In [13]:
df_tb_raw_tidy['faixa_idade'] = df_tb_raw_tidy['faixa'].map(criar_faixas, na_action='ignore')

In [14]:
df_tb_raw_tidy.drop(['faixa'], axis=1, inplace = True )

In [20]:
df_tb_raw_tidy.head()


,country,year,sexo_idade,num_casos,sexo,faixa_idade
0,AD,2000,m014,0.0,m,0-14
1,AE,2000,m014,2.0,m,0-14
2,AF,2000,m014,52.0,m,0-14
3,AG,2000,m014,0.0,m,0-14
4,AL,2000,m014,2.0,m,0-14


## Variáveis Armazenadas em Colunas e Linhas

Esse dataset apresenta os registros diários do tempo (temperaturas) na estação MX17004,no Mexico, pelo período de 5 meses em 2010.

Problemas:
* Variáveis armazenadas nas linhas (tmin, tmax) e também colunas (dias)

In [22]:
df_weather_raw = pd.read_csv('weather-raw.csv')

In [23]:
df_weather_raw.head()

,id,year,month,element,d1,d2,d3,d4,d5,d6,d7,d8
0,MX17004,2010,1,tmax,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,MX17004,2010,1,tmin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,MX17004,2010,2,tmax,NaN,27.3,24.1,NaN,NaN,NaN,NaN,NaN
3,MX17004,2010,2,tmin,NaN,14.4,14.4,NaN,NaN,NaN,NaN,NaN
4,MX17004,2010,3,tmax,NaN,NaN,NaN,NaN,32.1,NaN,NaN,NaN


Para transformar esse dataset precisaremos, a partir das colunas de dia e element (temperatura máxima e mínima). criar as variáveis data, temperatura_máxima e temperatura_mínima.

In [24]:
df_weather_tidy = pd.melt(df_weather_raw , id_vars = ['id' , 'year' , 'month' , 'element'] , var_name = 'dia' , value_name= 'temperatura')     

In [25]:
df_weather_tidy.head()

,id,year,month,element,dia,temperatura
0,MX17004,2010,1,tmax,d1,NaN
1,MX17004,2010,1,tmin,d1,NaN
2,MX17004,2010,2,tmax,d1,NaN
3,MX17004,2010,2,tmin,d1,NaN
4,MX17004,2010,3,tmax,d1,NaN


In [26]:
df_weather_tidy['dia_numero'] = df_weather_tidy["dia"].str.extract("(\d)")

In [27]:
df_weather_tidy['dia_numero'].head()

0    1
1    1
2    1
3    1
4    1
Name: dia_numero, dtype: object

Vamos agora criar uma data a partir do mês, ano e dia.

In [30]:
# Primeiramente converter para numerico tudo
df_weather_tidy.loc[:, ['year', 'month', 'dia_numero']] = df_weather_tidy.loc[:, ['year', 'month', 'dia_numero']].astype('int')


In [31]:
def criar_data(x):
    data = datetime.datetime(year=x['year'], month=int(x['month']), day=x['dia_numero'])
    return data

In [32]:
df_weather_tidy['data'] = df_weather_tidy.apply(criar_data, axis=1)

In [34]:
df_weather_tidy.head()

,id,year,month,element,dia,temperatura,dia_numero,data
0,MX17004,2010,1,tmax,d1,NaN,1,2010-01-01
1,MX17004,2010,1,tmin,d1,NaN,1,2010-01-01
2,MX17004,2010,2,tmax,d1,NaN,1,2010-02-01
3,MX17004,2010,2,tmin,d1,NaN,1,2010-02-01
4,MX17004,2010,3,tmax,d1,NaN,1,2010-03-01


In [35]:
df_weather_tidy.dropna(inplace=True)

# Joins

Joins são operações entre tabelas e servem para unir informações, combiando-as em uma única tabela. Os tipos de joins mais comuns são: left join, right join, inner join e outer join.

Para construir joins no pandas, temos duas formas: ```join``` e ```merge```. O método join funciona com indíces de data frame. O método merge, por sua vez, funciona com colunas, por isso iremos utilizá-lo.

Imagine que queiramos representar em tabelas os produtos de uma loja e as compras de clientes.

In [37]:
produtos=pd.DataFrame({
    'Produto_ID':[101,102,103,104,105,106,107],
    'nome_produto':['Relógio','Mochila','Tênis','Celular','Livro','Xampu','Computador'],
    'categoria':['estilo','estilo','estilo','eletrônicos','estudos','beleza','eletrônicos'],
    'preço':[299.0,1350.50,2999.0,14999.0,145.0,110.0,79999.0],
    'cidade_seller':['Delhi','Mumbai','Chennai','Kolkata','Delhi','Chennai','Bengalore']
})



In [38]:
clientes=pd.DataFrame({
    'id':[1,2,3,4,5,6,7,8,9],
    'name':['Ulrich','Jonas','Claudia','Adan','Eva','Helge','Elisabeth','Martha','Hannah'],
    'age':[20,25,15,10,30,65,35,18,23],
    'Produto_ID':[101,0,106,0,103,104,0,0,107],
    'item':['Relógio','NA','Xampu','NA','Tênis','Celular','NA','NA','Computador'],
    'cidade':['Mumbai','Delhi','Bangalore','Chennai','Chennai','Delhi','Kolkata','Delhi','Mumbai']
})

## Inner Join

Inner join é a operação que nos trará registros das duas tabelas que compartilham pelo menos uma informação em comum.

<img src="https://3.bp.blogspot.com/-MNEsPEb2v6I/UlATX9dcn4I/AAAAAAAAAH0/Jq2KuHPAJZM/s1600/INNER+JOIN.png" height=500 widht=239>

sintaxe: ```df1.merge(df2, left_on, right_on, how = )```

```df1```: dataframe 1 (left)<br>
```df2```: dataframe 2 (right)<br>
```left_on```: Coluna para dar match no df1<br>
```right_on```: Coluna para dar match no df2<br>
```how```: inner, right, left, outer

In [40]:
clientes.merge(produtos , left_on = 'Produto_ID' , right_on = 'Produto_ID' , how= 'inner')

,id,name,age,Produto_ID,item,cidade,nome_produto,categoria,preço,cidade_seller
0,1,Ulrich,20,101,Relógio,Mumbai,Relógio,estilo,299.0,Delhi
1,3,Claudia,15,106,Xampu,Bangalore,Xampu,beleza,110.0,Chennai
2,5,Eva,30,103,Tênis,Chennai,Tênis,estilo,2999.0,Chennai
3,6,Helge,65,104,Celular,Delhi,Celular,eletrônicos,14999.0,Kolkata
4,9,Hannah,23,107,Computador,Mumbai,Computador,eletrônicos,79999.0,Bengalore


In [41]:
clientes

,id,name,age,Produto_ID,item,cidade
0,1,Ulrich,20,101,Relógio,Mumbai
1,2,Jonas,25,0,NA,Delhi
2,3,Claudia,15,106,Xampu,Bangalore
3,4,Adan,10,0,NA,Chennai
4,5,Eva,30,103,Tênis,Chennai
5,6,Helge,65,104,Celular,Delhi
6,7,Elisabeth,35,0,NA,Kolkata
7,8,Martha,18,0,NA,Delhi
8,9,Hannah,23,107,Computador,Mumbai


## Left Join

Nem sempre há correspondência em todas as linhas das duas tabelas. Pode acontecer de um registro da tabela 1 não estar contido na tabela 2 e vice-versa. Mesmo assim, às vezes temos a necessidade de trazer todos os registros de uma tabela específica. No Left Join ele mantém os registros da tabela da esquerda.

<img src="https://upload.wikimedia.org/wikipedia/commons/f/f2/Left_JOIN.png" height=500 width=500>

In [44]:
clientes.merge(produtos , left_on = 'Produto_ID' , right_on = 'Produto_ID' , how = 'left')

,id,name,age,Produto_ID,item,cidade,nome_produto,categoria,preço,cidade_seller
0,1,Ulrich,20,101,Relógio,Mumbai,Relógio,estilo,299.0,Delhi
1,2,Jonas,25,0,NA,Delhi,NaN,NaN,NaN,NaN
2,3,Claudia,15,106,Xampu,Bangalore,Xampu,beleza,110.0,Chennai
3,4,Adan,10,0,NA,Chennai,NaN,NaN,NaN,NaN
4,5,Eva,30,103,Tênis,Chennai,Tênis,estilo,2999.0,Chennai
5,6,Helge,65,104,Celular,Delhi,Celular,eletrônicos,14999.0,Kolkata
6,7,Elisabeth,35,0,NA,Kolkata,NaN,NaN,NaN,NaN
7,8,Martha,18,0,NA,Delhi,NaN,NaN,NaN,NaN
8,9,Hannah,23,107,Computador,Mumbai,Computador,eletrônicos,79999.0,Bengalore


## Right Join

O Right Join é o contrário do Left Join. Ou seja, quando não houver correspondência, mesmo assim os registros da tabela da direita serão mantidos.

<img src="https://javatutorial.net/wp-content/uploads/2018/02/sql-right-join.png" height=300 width=530>

In [47]:
clientes.merge(produtos , left_on = 'Produto_ID' , right_on = 'Produto_ID' , how = 'right')

,id,name,age,Produto_ID,item,cidade,nome_produto,categoria,preço,cidade_seller
0,1.0,Ulrich,20.0,101,Relógio,Mumbai,Relógio,estilo,299.0,Delhi
1,NaN,NaN,NaN,102,NaN,NaN,Mochila,estilo,1350.5,Mumbai
2,5.0,Eva,30.0,103,Tênis,Chennai,Tênis,estilo,2999.0,Chennai
3,6.0,Helge,65.0,104,Celular,Delhi,Celular,eletrônicos,14999.0,Kolkata
4,NaN,NaN,NaN,105,NaN,NaN,Livro,estudos,145.0,Delhi
5,3.0,Claudia,15.0,106,Xampu,Bangalore,Xampu,beleza,110.0,Chennai
6,9.0,Hannah,23.0,107,Computador,Mumbai,Computador,eletrônicos,79999.0,Bengalore


## Outer Join

Por fim, o outer join traz informações das duas tabelas, mesmo sem correspondência de um lado ou de outro.

<img src="https://cdn.oracletutorial.com/wp-content/uploads/2019/02/Oracle-Joins-Full-Outer-Join.png" >

In [48]:
clientes.merge(produtos, left_on='Produto_ID', right_on='Produto_ID', how='outer')

,id,name,age,Produto_ID,item,cidade,nome_produto,categoria,preço,cidade_seller
0,1.0,Ulrich,20.0,101,Relógio,Mumbai,Relógio,estilo,299.0,Delhi
1,2.0,Jonas,25.0,0,NA,Delhi,NaN,NaN,NaN,NaN
2,4.0,Adan,10.0,0,NA,Chennai,NaN,NaN,NaN,NaN
3,7.0,Elisabeth,35.0,0,NA,Kolkata,NaN,NaN,NaN,NaN
4,8.0,Martha,18.0,0,NA,Delhi,NaN,NaN,NaN,NaN
5,3.0,Claudia,15.0,106,Xampu,Bangalore,Xampu,beleza,110.0,Chennai
6,5.0,Eva,30.0,103,Tênis,Chennai,Tênis,estilo,2999.0,Chennai
7,6.0,Helge,65.0,104,Celular,Delhi,Celular,eletrônicos,14999.0,Kolkata
8,9.0,Hannah,23.0,107,Computador,Mumbai,Computador,eletrônicos,79999.0,Bengalore
9,NaN,NaN,NaN,102,NaN,NaN,Mochila,estilo,1350.5,Mumbai
